#Importing the Necessary Libraries

In [1]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.8 MB/s eta 0:00:00


In [2]:
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BartForConditionalGeneration, BartTokenizer, T5ForConditionalGeneration, T5Tokenizer, PegasusForConditionalGeneration, PegasusTokenizer, ProphetNetForConditionalGeneration, ProphetNetTokenizer, pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#Multi-Model Abstractive Text Summarization

In [3]:
model1 = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer1 = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

model2 = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')
tokenizer2 = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')

model3 = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer3 = T5Tokenizer.from_pretrained('t5-small')

model4 = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
tokenizer4 = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

model5 = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased')
tokenizer5 = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [6]:
def abstractive_summarization(text):
    input_ids1 = tokenizer1.encode(text, return_tensors='pt')
    summary_ids1 = model1.generate(input_ids1, num_beams=4, length_penalty=2.0, max_length=1024, min_length=nw, early_stopping=True)
    summary1 = tokenizer1.decode(summary_ids1[0], skip_special_tokens=True)

    input_ids2 = tokenizer2.encode(text, return_tensors='pt')
    summary_ids2 = model2.generate(input_ids2, num_beams=4, length_penalty=2.0, max_length=1024, min_length=nw, early_stopping=True)
    summary2 = tokenizer2.decode(summary_ids2[0], skip_special_tokens=True)

    input_ids3 = tokenizer3.encode("summarize: " + text, return_tensors="pt")
    summary_ids3 = model3.generate(input_ids3, max_length=1024, min_length=nw, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary3 = tokenizer3.decode(summary_ids3[0], skip_special_tokens=True)

    input_ids4 = tokenizer4.encode(text, return_tensors="pt")
    summary_ids4 = model4.generate(input_ids4, max_length=1024, min_length=nw, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary4 = tokenizer4.decode(summary_ids4[0], skip_special_tokens=True)

    input_ids5 = tokenizer5.encode(text, return_tensors="pt")
    summary_ids5 = model5.generate(input_ids5, max_length=1024, min_length=nw, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary5 = tokenizer5.decode(summary_ids5[0], skip_special_tokens=True)

    combined_summary = f"\nSummary (BART): {summary1}\n \nSummary (DistilBART): {summary2}\n \nSummary (T5): {summary3}\n \nSummary (PEGASUS): {summary4}\n \nSummary (ProphetNet): {summary5}"
    return combined_summary

In [7]:
text = """
Extractive summarization is a text summarization technique that selects and extracts the most important sentences or phrases from the original text to create a summary. It does not involve generating new sentences or rephrasing the original text. Common methods for extractive summarization include TF-IDF, TextRank, and graph-based approaches.

TextRank is an extractive summarization technique that adapts Google's PageRank algorithm to score sentences based on their importance in the document. It represents the text as a graph, with sentences as nodes and edges based on relationships such as similarity. Using graph algorithms, TextRank identifies and selects the most important sentences for the summary.

TF-IDF (Term Frequency-Inverse Document Frequency) is another popular extractive summarization method that assigns scores to words based on their frequency in the document and the corpus. Sentences with higher scores, calculated based on the TF-IDF scores of their words, are selected for the summary.

In summary, extractive summarization involves selecting and extracting important sentences or phrases directly from the original text to create a condensed version while retaining key information.
"""

words = word_tokenize(text)
num_words = len(words)
nw = round(num_words / 2.5)

summary = abstractive_summarization(text)
print(summary)


Summary (BART): Extractive summarization is a text summarization technique that selects and extracts the most important sentences or phrases from the original text to create a summary. Common methods for extractive summarization include TF-IDF, TextRank, and graph-based approaches. It does not involve generating new sentences or rephrasing theOriginal text. It represents the text as a graph with sentences as nodes and edges based on relationships such as similarity.
 
Summary (DistilBART):  Common methods for extractive summarization include TF-IDF, TextRank, and graph-based approaches. TextRank scores sentences based on their importance in the document. TF-F (Term Frequency-Inverse Document Frequency) assigns scores to words based on frequency in document and corpus. Graph-based approach represents the text as a graph, with sentences as nodes and edges based on relationships such as similarity.
 
Summary (T5): extractive summarization is a text summarization technique that selects an

#Extractive Summarization using TextRank and TF-IDF

In [8]:
def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

def calculate_similarity_matrix(sentences, vectorizer):
    sentence_vectors = vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(sentence_vectors)
    return similarity_matrix

def generate_summary(text, num_sentences):
    sentences = preprocess_text(text)
    vectorizer = CountVectorizer().fit(sentences)
    similarity_matrix = calculate_similarity_matrix(sentences, vectorizer)

    scores = np.array(similarity_matrix.sum(axis=1)).flatten()
    ranked_sentences = [sentences[i] for i in scores.argsort()[-num_sentences:]]

    summary = ' '.join(ranked_sentences)
    return summary

In [9]:
text = """
Extractive summarization is a text summarization technique that selects and extracts the most important sentences or phrases from the original text to create a summary. It does not involve generating new sentences or rephrasing the original text. Common methods for extractive summarization include TF-IDF, TextRank, and graph-based approaches.

TextRank is an extractive summarization technique that adapts Google's PageRank algorithm to score sentences based on their importance in the document. It represents the text as a graph, with sentences as nodes and edges based on relationships such as similarity. Using graph algorithms, TextRank identifies and selects the most important sentences for the summary.

TF-IDF (Term Frequency-Inverse Document Frequency) is another popular extractive summarization method that assigns scores to words based on their frequency in the document and the corpus. Sentences with higher scores, calculated based on the TF-IDF scores of their words, are selected for the summary.

In summary, extractive summarization involves selecting and extracting important sentences or phrases directly from the original text to create a condensed version while retaining key information.
"""

sentences = sent_tokenize(text)
num_sentences = len(sentences)
ns= round(num_sentences/2)

summary = generate_summary(text, num_sentences=ns)
print("Summary:")
print(summary)

Summary:
TextRank is an extractive summarization technique that adapts Google's PageRank algorithm to score sentences based on their importance in the document. In summary, extractive summarization involves selecting and extracting important sentences or phrases directly from the original text to create a condensed version while retaining key information. Using graph algorithms, TextRank identifies and selects the most important sentences for the summary. 
Extractive summarization is a text summarization technique that selects and extracts the most important sentences or phrases from the original text to create a summary.


#Hybrid Extractive-Abstractive Text Summarization

In [12]:
def extractive_summarization(text, num_sentences):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalnum() and word.lower() not in stopwords.words("english")]
    word_freq = nltk.FreqDist(words)

    sentence_scores = {}
    for sentence in sentences:
        for word in nltk.word_tokenize(sentence.lower()):
            if word in word_freq.keys():
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = word_freq[word]
                else:
                    sentence_scores[sentence] += word_freq[word]

    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    return ' '.join(summary_sentences)

def abstractive_summarization(text):
    model_name = "facebook/bart-large-cnn"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt")

    summary_ids = model.generate(inputs, max_length=1024, min_length=nw, length_penalty=2.0, num_beams=10)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

def summarization(text, num_sentences=ns):
    extractive_summary = extractive_summarization(text, num_sentences)
    abstractive_summary = abstractive_summarization(extractive_summary)

    return abstractive_summary

In [13]:
text = """
Extractive summarization is a text summarization technique that selects and extracts the most important sentences or phrases from the original text to create a summary. It does not involve generating new sentences or rephrasing the original text. Common methods for extractive summarization include TF-IDF, TextRank, and graph-based approaches.

TextRank is an extractive summarization technique that adapts Google's PageRank algorithm to score sentences based on their importance in the document. It represents the text as a graph, with sentences as nodes and edges based on relationships such as similarity. Using graph algorithms, TextRank identifies and selects the most important sentences for the summary.

TF-IDF (Term Frequency-Inverse Document Frequency) is another popular extractive summarization method that assigns scores to words based on their frequency in the document and the corpus. Sentences with higher scores, calculated based on the TF-IDF scores of their words, are selected for the summary.

In summary, extractive summarization involves selecting and extracting important sentences or phrases directly from the original text to create a condensed version while retaining key information.
"""

sentences = sent_tokenize(text)
words = word_tokenize(text)

num_sentences = len(sentences)
num_words = len(words)

ns= round(num_sentences/2)
nw= round(num_words/2.5)


hybrid_summary = summarization(text)
print("Hybrid Summary:")
print(hybrid_summary)

Hybrid Summary:
Extractive summarization involves selecting and extracting important sentences or phrases directly from the original text. TextRank is an extractive summarization technique that adapts Google's PageRank algorithm to score sentences based on their importance in the document. TF-IDF (Term Frequency-Inverse Document Frequency) is another popular extractive summarizes method that assigns scores to words based on frequency in document and corpus.


#Ensemble Extractive-Abstractive Summarization

In [3]:
model1 = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer1 = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

model2 = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')
tokenizer2 = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')

model3 = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
tokenizer3 = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

model4 = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased')
tokenizer4 = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased')

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
def extractive_summarization_combined(summary, num_sentences):
    sentences = sent_tokenize(summary)
    words = word_tokenize(summary)
    words = [word.lower() for word in words if word.isalnum() and word.lower() not in stopwords.words("english")]
    word_freq = nltk.FreqDist(words)

    sentence_scores = {}
    for sentence in sentences:
        for word in nltk.word_tokenize(sentence.lower()):
            if word in word_freq.keys():
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = word_freq[word]
                else:
                    sentence_scores[sentence] += word_freq[word]

    final_summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    return ' '.join(final_summary_sentences)

def summarize_text_ensemble(text):
    input_ids1 = tokenizer1.encode(text, return_tensors='pt')
    summary_ids1 = model1.generate(input_ids1, num_beams=4, length_penalty=2.0, max_length=1024, min_length=nw, early_stopping=True)
    summary1 = tokenizer1.decode(summary_ids1[0], skip_special_tokens=True)

    input_ids2 = tokenizer2.encode(text, return_tensors='pt')
    summary_ids2 = model2.generate(input_ids2, num_beams=4, length_penalty=2.0, max_length=1024, min_length=nw, early_stopping=True)
    summary2 = tokenizer2.decode(summary_ids2[0], skip_special_tokens=True)


    input_ids3 = tokenizer3.encode(text, return_tensors="pt")
    summary_ids3 = model3.generate(input_ids3, max_length=1024, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary3 = tokenizer3.decode(summary_ids3[0], skip_special_tokens=True)


    input_ids4 = tokenizer4.encode(text, return_tensors="pt")
    summary_ids4 = model4.generate(input_ids4, max_length=1024, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary4 = tokenizer4.decode(summary_ids4[0], skip_special_tokens=True)

    combined_summary = summary1 + ' ' + summary2 + ' ' + summary3 + ' ' + summary4
    final_summary = extractive_summarization_combined(combined_summary, num_sentences=ns)

    return final_summary

In [5]:
text = """
Extractive summarization is a text summarization technique that selects and extracts the most important sentences or phrases from the original text to create a summary. It does not involve generating new sentences or rephrasing the original text. Common methods for extractive summarization include TF-IDF, TextRank, and graph-based approaches.

TextRank is an extractive summarization technique that adapts Google's PageRank algorithm to score sentences based on their importance in the document. It represents the text as a graph, with sentences as nodes and edges based on relationships such as similarity. Using graph algorithms, TextRank identifies and selects the most important sentences for the summary.

TF-IDF (Term Frequency-Inverse Document Frequency) is another popular extractive summarization method that assigns scores to words based on their frequency in the document and the corpus. Sentences with higher scores, calculated based on the TF-IDF scores of their words, are selected for the summary.

In summary, extractive summarization involves selecting and extracting important sentences or phrases directly from the original text to create a condensed version while retaining key information.
"""

words = word_tokenize(text)
num_words = len(words)
nw = round(num_words / 2.5)

sentences = nltk.sent_tokenize(text)
num_sentences = len(sentences)
ns= round(num_sentences/2)

summary = summarize_text_ensemble(text)

print("Ensemble Summary:")
print(summary)

Ensemble Summary:
Extractive summarization is a text summarization technique that selects and extracts the most important sentences or phrases from the original text to create a summary. Common methods for extractive summarization include TF-IDF, TextRank, and graph-based approaches. extractive summarization does not involve creating new sentences or phrases from the original text. Graph-based approach represents the text as a graph, with sentences as nodes and edges based on relationships such as similarity.
